In [38]:
!pip install implicit

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import copy

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

**Ответ.** Товар 999999 мы использовали как замену наименее популярным товарам. Он нужен, чтобы уменьшить число товаров для обучения, т.е. мы незначительно жертвуем точностью ради скорости обучения и предикта. Кроме того увеличивается плотность матрицы user-item, т.к. становится меньше доля нулей.

Качество рекомендаций становится немного хуже и также этот товар-заглушка может выдаваться как рекомендуемый на первых позициях просто потому, что его много будет.

Удалить его можно, но это приведет к потере информации и появлению части юзеров у которых не будет покупок.

In [41]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [42]:
def compare_models_precision_at_k(user_id, result_data, k=5):
    itemitem_dummy_droped_pre = precision_at_k(result_data['itemitem_dummy_droped'][user_id], result_data['actual'][user_id], k)
    itemitem_pre = precision_at_k(result_data['itemitem'][user_id], result_data['actual'][user_id], k)
    return itemitem_pre, itemitem_dummy_droped_pre

In [43]:
data = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/recommender_systems/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [44]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [45]:
result_new = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_new.columns=['user_id', 'actual']
result_new['actual'] = result_new['actual'].apply(lambda x: list(x))
result_new.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [46]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [47]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [48]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix (позволяет в сжатом виде хранить большие матрицы, где много нулей)
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,819255,819304,819308,819330,819518,819594,819643,819765,819840,819845,819927,819978,820082,820122,820165,820291,820301,820321,820361,820486,820518,820560,820701,820895,821025,821083,821200,821209,821219,821344,...,13512965,13671759,13672065,13777104,13841744,13842088,13842090,13842214,13842224,13877192,13945141,13945244,13987135,14025185,14043817,14043823,14043825,14043826,14050436,14050460,14050461,14077333,14106445,15452677,15452812,15506577,15511891,15596279,15596488,15596515,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [50]:
user_item_matrix_dummy_droped = user_item_matrix.drop(columns=[999999])

sparse_user_item_dummy_droped = csr_matrix(user_item_matrix_dummy_droped).tocsr()
999999 in user_item_matrix_dummy_droped.columns  # Проверка правильности удаления 999999

False

In [51]:
# перенумеруем пользователей и товары
userids_dummy_droped = user_item_matrix_dummy_droped.index.values
itemids_dummy_droped = user_item_matrix_dummy_droped.columns.values

matrix_userids_dummy_droped = np.arange(len(userids_dummy_droped))
matrix_itemids_dummy_droped = np.arange(len(itemids_dummy_droped))

id_to_itemid_dummy_droped = dict(zip(matrix_itemids_dummy_droped, itemids_dummy_droped))
id_to_userid_dummy_droped = dict(zip(matrix_userids_dummy_droped, userids_dummy_droped))

itemid_to_id_dummy_droped = dict(zip(itemids_dummy_droped, matrix_itemids_dummy_droped))
userid_to_id_dummy_droped = dict(zip(userids_dummy_droped, matrix_userids_dummy_droped))

In [52]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

model_dummy_droped = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model_dummy_droped.fit(csr_matrix(user_item_matrix_dummy_droped).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 4.2 s, sys: 51.5 ms, total: 4.25 s
Wall time: 3.07 s


In [53]:
%%time

result_new['itemitem'] = result_new['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

result_new['itemitem_dummy_droped'] = result_new['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_dummy_droped.recommend(userid=userid_to_id_dummy_droped[x], 
                                    user_items=sparse_user_item,   # на вход для рекомендаций даём user-item matrix полную с товаром 999999, тогда как обучались без 999999 !!
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 129 ms, sys: 952 µs, total: 130 ms
Wall time: 131 ms


In [54]:
result_new.head(10)

,user_id,actual,itemitem,itemitem_dummy_droped
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[999999, 1082185, 981760, 1127831, 995242]","[1082172, 981760, 1127825, 995242, 1097849]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[999999, 1082185, 981760, 1098066, 995242]","[1082172, 981760, 1097849, 826249, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[999999, 1082185, 981760, 1127831, 995242]","[1082172, 981760, 1127825, 1097849, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[999999, 1082185, 981760, 1127831, 995242]","[1082172, 981760, 1097849, 1127825, 995242]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[999999, 1082185, 981760, 1127831, 1098066]","[1082172, 981760, 1097849, 826249, 995242]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[999999, 1082185, 981760, 995242, 1098066]","[1082172, 981760, 1097849, 995242, 1127825]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[999999, 1082185, 981760, 1098066, 1127831]","[1082172, 981760, 1097849, 826249, 995242]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[999999, 1082185, 981760, 1098066, 995242]","[1082172, 981760, 1097849, 826249, 995242]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[999999, 1082185, 981760, 1127831, 995242]","[1082172, 981760, 1127825, 995242, 840361]"
9,16,"[1062973, 1082185, 13007710]","[999999, 1082185, 981760, 995242, 1029743]","[1082172, 981760, 866227, 840361, 995242]"


In [55]:
for column in result_new.columns[2:]:
    print(f"Mean precision@5 for {column} is: {result_new.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean()}")

Mean precision@5 for itemitem is: 0.13692458374142857
Mean precision@5 for itemitem_dummy_droped is: 0.07061704211557317


Точность значительно ухудшилась после отбрасывания товара-заглушки.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорционально логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

*Пример*  
item_1 - 5, item_2 - 7, item_3 - 4  # / sum  
item_1 - 5 / 16, item_2 - 7 / 16, item_3 - 4 / 16

In [56]:
def weighted_random_recommendation(items_weights, n=5):
    """Взвешенные случайные рекоммендации на основе доли выбранного показателя по item в общей сумме по данному показателю.
    На вход принимается сгрупированная по item таблица относительно целевого показателя."""
    
    items = np.array(items_weights['item_id'])

    items_weights_sum = items_weights['weights'].sum()  # Считаю общую сумму 1 раз, чтобы не вычислять на всей длине выборки
    probabilities = np.array(items_weights['weights'].apply(lambda x: x / items_weights_sum))  # Нормирую каждое значение на общую сумму, чтобы получить вероятности

    recs = np.random.choice(items, size=n, p=probabilities, replace=False)
    
    return recs.tolist()

Сделайте предсказания

In [57]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights.columns = ['item_id', 'weights']

result_new['weighted_random_recommendation_by_sales_value'] = result_new['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

CPU times: user 5.03 s, sys: 13.2 ms, total: 5.04 s
Wall time: 5.06 s


In [58]:
%%time

items_weights = data_train.groupby('item_id')['quantity'].sum().reset_index()
items_weights.columns = ['item_id', 'weights']

result_new['weighted_random_recommendation_by_quantity'] = result_new['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

CPU times: user 6.58 s, sys: 4.42 ms, total: 6.58 s
Wall time: 6.62 s


In [59]:
result_new.head(5)

,user_id,actual,itemitem,itemitem_dummy_droped,weighted_random_recommendation_by_sales_value,weighted_random_recommendation_by_quantity
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[999999, 1082185, 981760, 1127831, 995242]","[1082172, 981760, 1127825, 995242, 1097849]","[999999, 6633273, 6534178, 8090537, 827261]","[6534178, 6533889, 6544236, 6534166, 397896]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[999999, 1082185, 981760, 1098066, 995242]","[1082172, 981760, 1097849, 826249, 995242]","[999999, 960984, 1076738, 9245106, 882288]","[6534178, 480014, 6534166, 5668996, 6533889]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[999999, 1082185, 981760, 1127831, 995242]","[1082172, 981760, 1127825, 1097849, 995242]","[999999, 1062966, 883068, 1087916, 6534178]","[6534178, 6534166, 1388206, 9245106, 6533889]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[999999, 1082185, 981760, 1127831, 995242]","[1082172, 981760, 1097849, 1127825, 995242]","[962229, 936355, 944836, 999999, 8067128]","[6534178, 6533889, 6534166, 5668996, 1426702]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[999999, 1082185, 981760, 1127831, 1098066]","[1082172, 981760, 1097849, 826249, 995242]","[999999, 6034648, 1058214, 978983, 849202]","[6534166, 6534178, 397896, 1404121, 1022003]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [60]:
result_weighted = result_new.drop(columns=['actual', 'itemitem'])

In [61]:
result = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/recommender_systems/data/preds.csv')  # загрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15927063, 905162, 6034968, 10311367, 941944]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1122732, 983151, 6552927, 257927, 1074209]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [62]:
type(result['actual'][0])

str

In [63]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result[a] =  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [64]:
# Прикрепляю новые расчеты
result = pd.merge(result, result_weighted, on="user_id")

In [65]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_dummy_droped,weighted_random_recommendation_by_sales_value,weighted_random_recommendation_by_quantity
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15927063, 905162, 6034968, 10311367, 941944]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082172, 981760, 1127825, 995242, 1097849]","[999999, 6633273, 6534178, 8090537, 827261]","[6534178, 6533889, 6544236, 6534166, 397896]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1122732, 983151, 6552927, 257927, 1074209]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082172, 981760, 1097849, 826249, 995242]","[999999, 960984, 1076738, 9245106, 882288]","[6534178, 480014, 6534166, 5668996, 6533889]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[969846, 866285, 10122659, 9485565, 6442626]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[1082172, 981760, 1127825, 1097849, 995242]","[999999, 1062966, 883068, 1087916, 6534178]","[6534178, 6534166, 1388206, 9245106, 6533889]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5995778, 10311808, 7152900, 8069231, 1010481]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]","[1082172, 981760, 1097849, 1127825, 995242]","[962229, 936355, 944836, 999999, 8067128]","[6534178, 6533889, 6534166, 5668996, 1426702]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1032838, 769086, 15828354, 1081434, 972437]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[1082172, 981760, 1097849, 826249, 995242]","[999999, 6034648, 1058214, 978983, 849202]","[6534166, 6534178, 397896, 1404121, 1022003]"


In [66]:
#result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()

In [67]:
for column in result.columns[2:]:
    print(f"Mean precision@5 for {column} is: {result.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean()}")

Mean precision@5 for random_recommendation is: 0.0002938295788442704
Mean precision@5 for popular_recommendation is: 0.15523996082272082
Mean precision@5 for itemitem is: 0.13692458374142857
Mean precision@5 for cosine is: 0.13290891283055686
Mean precision@5 for tfidf is: 0.1389813907933383
Mean precision@5 for own_purchases is: 0.17969311132876015
Mean precision@5 for itemitem_dummy_droped is: 0.07061704211557317
Mean precision@5 for weighted_random_recommendation_by_sales_value is: 0.03046033300685615
Mean precision@5 for weighted_random_recommendation_by_quantity is: 0.047404505386875975


Хуже всего работают случайные алгоритмы, но взвешенные оказываются относительно лучше полного рандома. Алгоритмы itemitem, cosine, tfidf - показали примерно одинаковую эффективность, но хуже бейзлайнов по популярности и собственным покупкам. Связываю это с тем, что данные бейзлайны завышают значение dummy-товара 999999, который мы и не будем рекомендовать.

Вижу один из вариантов решения такой: выбирать алгоритмами 6 рекомендаций, а потом убирать от туда 999999. Так бы оставалось 5 валидных рекомендаций без dummy-товара. И на них же и сравнивать precision@k.

Проверю гипотезу. Представим, что нужно было получить 4 рекомендации (чтобы не пересчитывать все алгоритмы). Для эксперимента я выкину 999999 из текущего файла рекомендаций и пересчитаю precision@4, чтобы посмотреть изменится ли относительное качество алгоритмов при таком подходе.

In [68]:
result.to_csv(r'/content/drive/MyDrive/Colab Notebooks/recommender_systems/data/preds_new.csv', index=False)

In [70]:
result_4 = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/recommender_systems/data/preds_new.csv')
for a in result_4.columns[1:]:
    result_4[a] =  result_4[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))
result_4.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_dummy_droped,weighted_random_recommendation_by_sales_value,weighted_random_recommendation_by_quantity
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15927063, 905162, 6034968, 10311367, 941944]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082172, 981760, 1127825, 995242, 1097849]","[999999, 6633273, 6534178, 8090537, 827261]","[6534178, 6533889, 6544236, 6534166, 397896]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1122732, 983151, 6552927, 257927, 1074209]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082172, 981760, 1097849, 826249, 995242]","[999999, 960984, 1076738, 9245106, 882288]","[6534178, 480014, 6534166, 5668996, 6533889]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[969846, 866285, 10122659, 9485565, 6442626]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[1082172, 981760, 1127825, 1097849, 995242]","[999999, 1062966, 883068, 1087916, 6534178]","[6534178, 6534166, 1388206, 9245106, 6533889]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[5995778, 10311808, 7152900, 8069231, 1010481]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]","[1082172, 981760, 1097849, 1127825, 995242]","[962229, 936355, 944836, 999999, 8067128]","[6534178, 6533889, 6534166, 5668996, 1426702]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1032838, 769086, 15828354, 1081434, 972437]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[1082172, 981760, 1097849, 826249, 995242]","[999999, 6034648, 1058214, 978983, 849202]","[6534166, 6534178, 397896, 1404121, 1022003]"


In [71]:
for column in result_4.columns[2:]:
    result_4[column].apply(lambda x: x.pop(-1) if (999999 not in x) and (len(x) == 5) else x)
    result_4[column].apply(lambda x: x.remove(999999) if 999999 in x else x)

In [72]:
for column in result_4.columns[2:]:
    print(f"Mean precision@5 for {column} is: {result_4.apply(lambda x: precision_at_k(x[column], x['actual'],  4), axis=1).mean()}")

Mean precision@5 for random_recommendation is: 0.0002448579823702253
Mean precision@5 for popular_recommendation is: 0.10700293829578844
Mean precision@5 for itemitem is: 0.17115572967678747
Mean precision@5 for cosine is: 0.16613614103819785
Mean precision@5 for tfidf is: 0.17115572967678747
Mean precision@5 for own_purchases is: 0.22554080629301868


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in long_scalars
  del sys.path[0]


Mean precision@5 for itemitem_dummy_droped is: 0.0648873653281097
Mean precision@5 for weighted_random_recommendation_by_sales_value is: 0.037830558276199806
Mean precision@5 for weighted_random_recommendation_by_quantity is: 0.05754162585700294


Действительно, в итоге качество popular recomendation ухудшилось, т.к. мы убрали самый популярный dummy-товар. Алгоритмы, работающие на расстояниях, показали результат лучше всех бейзлайнов, кроме own_purchases.

Это печально на первый взгляд :) 

Но значит ли это, что они хуже в данном кейсе? Скорее всего - нет. У нас датасет по продуктовому ритейлу. Для него характерны повторные покупки регулярных товаров, таких как хлеб, молоко, яйца, и т.п. Поэтому успех на тестовой выборке модели own_purchases только говорит о том, что пользователи верны своим предпочтениям и часть их продуктовой корзины повторяется. Наша же задача предложить им что-то новое, что увеличит чек и расширит продуктовый набор. Поэтому правильно будет смотреть качество моделей itemitem, cosine, tfidf уже после релиза.

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [74]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,202291,35911
1,397896,1214994
2,420647,168661
3,480014,371107
4,545926,20134


In [75]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
top_5000[:5]

[6534178, 6533889, 6534166, 6544236, 1404121]

In [76]:
data_train_top5000_items = data_train.loc[data_train['item_id'].isin(top_5000), :]

In [77]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [78]:
%%time

items = data_train_top5000_items.item_id.unique()

result['random_recommendation_top5000_items'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 267 ms, sys: 0 ns, total: 267 ms
Wall time: 269 ms


In [ ]:
for column in result.columns[2:]:
    print(f"Mean precision@5 for {column} is: {result.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean()}")

**Вывод:** Рандом на топ-5000 популярных товаров даёт качество чуть лучше, чем на всей выборке, что ожидаемо.

In [79]:
%%time

model = ItemItemRecommender(K=10, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['itemitem_k_10'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


CPU times: user 2.28 s, sys: 24 ms, total: 2.31 s
Wall time: 1.77 s


In [80]:
%%time

model = ItemItemRecommender(K=20, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['itemitem_k_20'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


CPU times: user 2.41 s, sys: 26.3 ms, total: 2.44 s
Wall time: 1.8 s


In [81]:
%%time

model = ItemItemRecommender(K=30, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['itemitem_k_30'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


CPU times: user 2.48 s, sys: 19.4 ms, total: 2.5 s
Wall time: 1.81 s


In [82]:
%%time

model = CosineRecommender(K=10, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['cosine_k_10'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


CPU times: user 2.3 s, sys: 32.8 ms, total: 2.33 s
Wall time: 1.77 s


In [83]:
%%time

model = CosineRecommender(K=20, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['cosine_k_20'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


CPU times: user 2.4 s, sys: 36.8 ms, total: 2.44 s
Wall time: 1.87 s


In [84]:
%%time

model = CosineRecommender(K=30, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['cosine_k_30'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])


CPU times: user 2.45 s, sys: 30.3 ms, total: 2.48 s
Wall time: 1.87 s


In [85]:
%%time

model = TFIDFRecommender(K=10, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['tfidf_k_10'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])


CPU times: user 2.3 s, sys: 31.7 ms, total: 2.34 s
Wall time: 1.72 s


In [86]:
%%time

model = TFIDFRecommender(K=20, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['tfidf_k_20'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])


CPU times: user 2.38 s, sys: 31.2 ms, total: 2.42 s
Wall time: 1.82 s


In [87]:
%%time

model = TFIDFRecommender(K=30, num_threads=4) # K - кол-во билжайших соседей, по умолчанию используется 20, но мы ранее брали 5

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['tfidf_k_30'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])


CPU times: user 2.45 s, sys: 29.3 ms, total: 2.48 s
Wall time: 1.8 s


In [88]:
for column in result.columns[2:]:
    print(f"Mean precision@5 for {column} is: {result.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean()}")

Mean precision@5 for random_recommendation is: 0.0002938295788442704
Mean precision@5 for popular_recommendation is: 0.15523996082272082
Mean precision@5 for itemitem is: 0.13692458374142857
Mean precision@5 for cosine is: 0.13290891283055686
Mean precision@5 for tfidf is: 0.1389813907933383
Mean precision@5 for own_purchases is: 0.17969311132876015
Mean precision@5 for itemitem_dummy_droped is: 0.07061704211557317
Mean precision@5 for weighted_random_recommendation_by_sales_value is: 0.03046033300685615
Mean precision@5 for weighted_random_recommendation_by_quantity is: 0.047404505386875975
Mean precision@5 for random_recommendation_top5000_items is: 0.004897159647404504
Mean precision@5 for itemitem_k_10 is: 0.15093046033300497
Mean precision@5 for itemitem_k_20 is: 0.15259549461312247
Mean precision@5 for itemitem_k_30 is: 0.14946131243878374
Mean precision@5 for cosine_k_10 is: 0.1320274240940242
Mean precision@5 for cosine_k_20 is: 0.1366307541625843
Mean precision@5 for cosine_k_

**Вывод:** подбор числа k-ближайших соседей позволяет сильно улучшить модели.

In [107]:
# Попробуем разные куски от разных типов рекомендаций
result['popular'] = result['popular_recommendation'].apply(lambda x: x[:2])
result['own'] = result['own_purchases'].apply(lambda x: x[2:])
result['Popular_and_own'] = result['popular'] + result['own']
result = result.drop(columns=['popular', 'own'])

result['popular'] = result['popular_recommendation'].apply(lambda x: x[2:])
result['own'] = result['own_purchases'].apply(lambda x: x[:2])
result['Own_and_popular'] = result['own'] + result['popular']
result = result.drop(columns=['own', 'popular'])

result['ii_k20'] = result['itemitem_k_20'].apply(lambda x: x[:2])
result['tf_k30'] = result['tfidf_k_30'].apply(lambda x: x[2:])
result['ii_k20_tf_k30'] = result['ii_k20'] + result['tf_k30']
result = result.drop(columns=['ii_k20', 'tf_k30'])

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem_dummy_droped,weighted_random_recommendation_by_sales_value,weighted_random_recommendation_by_quantity,random_recommendation_top5000_items,itemitem_k_10,itemitem_k_20,itemitem_k_30,cosine_k_10,cosine_k_20,cosine_k_30,tfidf_k_10,tfidf_k_20,tfidf_k_30,Popular_and_own,Own_and_popular,ii_k20_tf_k30
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15927063, 905162, 6034968, 10311367, 941944]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1082172, 981760, 1127825, 995242, 1097849]","[999999, 6633273, 6534178, 8090537, 827261]","[6534178, 6533889, 6544236, 6534166, 397896]","[863506, 10180324, 913144, 848669, 1082185]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 1127831]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 999999, 981760, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 981760, 1127831, 999999, 840361]","[1082185, 981760, 1127831, 840361, 961554]","[6534178, 6533889, 1029743, 995785, 1004906]","[999999, 1082185, 1029743, 6534166, 1082185]","[999999, 1082185, 1127831, 840361, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1122732, 983151, 6552927, 257927, 1074209]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1082172, 981760, 1097849, 826249, 995242]","[999999, 960984, 1076738, 9245106, 882288]","[6534178, 480014, 6534166, 5668996, 6533889]","[837579, 8119157, 907014, 1131312, 1051211]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 1098066, 826249, 981760, 999999]","[1098066, 1082185, 826249, 981760, 999999]","[1082185, 1098066, 981760, 826249, 1044078]","[1082185, 981760, 826249, 1098066, 999999]","[1082185, 1098066, 826249, 981760, 1044078]","[6534178, 6533889, 1098066, 6534178, 1127831]","[999999, 1082185, 1029743, 6534166, 1082185]","[999999, 1082185, 826249, 981760, 1044078]"


In [108]:
for column in result.columns[2:]:
    print(f"Mean precision@5 for {column} is: {result.apply(lambda x: precision_at_k(x[column], x['actual'],  5), axis=1).mean()}")

Mean precision@5 for random_recommendation is: 0.0002938295788442704
Mean precision@5 for popular_recommendation is: 0.15523996082272082
Mean precision@5 for itemitem is: 0.13692458374142857
Mean precision@5 for cosine is: 0.13290891283055686
Mean precision@5 for tfidf is: 0.1389813907933383
Mean precision@5 for own_purchases is: 0.17969311132876015
Mean precision@5 for itemitem_dummy_droped is: 0.07061704211557317
Mean precision@5 for weighted_random_recommendation_by_sales_value is: 0.03046033300685615
Mean precision@5 for weighted_random_recommendation_by_quantity is: 0.047404505386875975
Mean precision@5 for random_recommendation_top5000_items is: 0.004897159647404504
Mean precision@5 for itemitem_k_10 is: 0.15093046033300497
Mean precision@5 for itemitem_k_20 is: 0.15259549461312247
Mean precision@5 for itemitem_k_30 is: 0.14946131243878374
Mean precision@5 for cosine_k_10 is: 0.1320274240940242
Mean precision@5 for cosine_k_20 is: 0.1366307541625843
Mean precision@5 for cosine_k_

**Вывод:** если так топроно брать куски разных решений, то качество получается хуже, чем у них в отдельности.

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.